# Transformers library
We will take advantage of custom transformers. This way, it is easy to use them in pipelines both for training and testing. We can easily select a subset of transformations and vary transformations based on used model.

In [31]:
# Local
from ipynb.fs.full.data_loader import load_train_test_all_cols_data

# Math and data stuff
import pandas as pd
import numpy as np

# Sklearn
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer

# Language stuff
from pymagnitude import Magnitude, MagnitudeUtils
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download("stopwords", quiet=True)

# Other
import html

---

In [2]:
train_X, test_X, train_y, test_y = load_train_test_all_cols_data()

## DenseTransformer

In [3]:
class DenseTransformer(TransformerMixin):
    """
    Makes sparse arrays dense.
    """
    def fit(self, X, y=None, **fit_params):
        return self

    def transform(self, X, y=None, **fit_params):
        return X.todense() # toarray() works too

---

## FeatureSelector

In [4]:
class FeatureSelector(BaseEstimator, TransformerMixin):
    """
    Selects a subset of features based on column names or data type.
    If both `columns` and `dtype_include` are given, union of selected columns is returned.
    """
    def __init__(self, columns=[], dtype_include=None):
        self.columns = columns
        self.dtype_include = dtype_include

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        X = X.copy()
        if self.dtype_include is not None:
            self.columns.extend(X.select_dtypes(self.dtype_include).columns)
        return X[set(self.columns)]

# DEMO 1
transformed = FeatureSelector(["AUTHOR", "DATE"]).transform(train_X)
transformed.head()

,DATE,AUTHOR
0,2015-05-22 11:46:35.988,Sonny Carter
1,2013-09-09 17:34:07.052,Lizzy Molly
2,2015-05-26 02:27:43.254,Warcorpse666
3,2014-09-03 16:32:59.000,Santeri Saariokari
4,2014-11-06 19:50:16.000,Quinho Divulgaçoes


In [5]:
# DEMO 2
transformed = FeatureSelector(dtype_include="string").transform(train_X)
transformed.head()

,COMMENT_ID,AUTHOR,CONTENT,INTERPRET
0,z13wxtdpeznid12et23ogtd4zoyvzbnoz04,Sonny Carter,I love this song sooooooooooooooo much﻿,eminem
1,_2viQ_Qnc6-adCzTDLAhqNVQ5hFYcjPyPI5m7pHY4BY,Lizzy Molly,PLEASE CHECK OUT MY VIDEO CALLED &quot;WE LOVE...,shakira
2,z13uutbriumnuj3rq04ccbvqlwjuj1srhyk0k,Warcorpse666,sorry but eminmem is a worthless wife beating ...,eminem
3,z121gbuy2unhc5m4n04cf3kyslqhepeqgvo0k,Santeri Saariokari,"Hey guys go to check my video name ""growtopia ...",katy
4,z133hdqrqpukup0lp22chhoaztrhvxov5,Quinho Divulgaçoes,me segue ha https://www.facebook.com/marcos.s...,katy


In [6]:
# DEMO 3
transformed = FeatureSelector(columns=["INTERPRET", "AUTHOR"], dtype_include="datetime").transform(train_X)
transformed.head()

,DATE,INTERPRET,AUTHOR
0,2015-05-22 11:46:35.988,eminem,Sonny Carter
1,2013-09-09 17:34:07.052,shakira,Lizzy Molly
2,2015-05-26 02:27:43.254,eminem,Warcorpse666
3,2014-09-03 16:32:59.000,katy,Santeri Saariokari
4,2014-11-06 19:50:16.000,katy,Quinho Divulgaçoes


---

## WordEmbeddings

In [7]:
class WordEmbeddingsSeries(BaseEstimator, TransformerMixin):
    """
    ! Works with Series, not DataFrame !
    For each row in series, which contains a sentence, 
    it embeds the words in the series into 300 dimensional vectors (one vector for each word).
    """
    def __init__(self, vectors):
        super().__init__()
        self.vectors = vectors

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        X = X.copy()
        tokenized = [word_tokenize(line) for line in X]
        return self.vectors.query(tokenized)
    
# DEMO
vectors = Magnitude(MagnitudeUtils.download_model('fasttext/medium/wiki-news-300d-1M'), pad_to_length=30)
transformed = WordEmbeddingsSeries(vectors).transform(train_X["CONTENT"][:20])
transformed.shape

(20, 30, 300)

---

## WordEmbeddingsDF

In [8]:
class WordEmbeddingsDF(BaseEstimator, TransformerMixin):
    """
    For each column (given in constructor or all columns if not specified),
    computes word embedddings of values in the column split with nltk.word_tokenize.
    The mean of these word embeddings is then computed, giving 300 dimensional vector for each row.
    This vector is then appended to the dataframe as 300 new columns.
    """
    def __init__(self, columns=None):
        super().__init__()
        self.columns = columns

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        X = X.copy()
        i = 0
        # encode all columns by default
        if self.columns is None:
            self.columns = X.columns

        for col in self.columns:
            tokenized = [word_tokenize(line) for line in X[col]]
            mean = np.mean(vectors.query(tokenized), axis=1)
            for j in range(300):
                X[f"{i}_EMBEDDED"] = mean[:,j]
                i += 1
                
        return X
    
transformed = WordEmbeddingsDF().transform(train_X[["AUTHOR", "CONTENT"]])
transformed.shape

(1249, 602)

---

## ExplorativeTransformer

In [9]:
class ExplorativeTransformer(BaseEstimator, TransformerMixin):
    """
    
    """
    def __init__(self):
        super().__init__()
        self.duplicated_and_spammers = 0
        self.duplicated_and_not_spammers = 0
        self.not_unique_authors = set()

    def fit(self, X, y=None):
        X = X.copy()
        
        stop = stopwords.words('english')

        author_set = set()
        for author in X["AUTHOR"]:
            if author in author_set:
                self.not_unique_authors.add(author)
            author_set.add(author)
            
        # are all duplicated authors spammers?
        for index, row in X.iterrows():
            if row["AUTHOR"] in self.not_unique_authors and y[index] == 1:
                self.duplicated_and_spammers += 1
            if row["AUTHOR"] in self.not_unique_authors and y[index] == 0:
                self.duplicated_and_not_spammers += 1
        
        """
        Creates 2 dictionaries: spam_dic and ham_dict, one that counts occurence of each word in spam comments, 
        other counts occurence of each word in ham comments.
        Further, only those words that are repeated more than 15 times are left in ham_dict, 
        and those repeated more than 35 times in spam_dict.
        Numbers are chosen like that, because I wanted to empahsise, that it is more importatnt to not mark as spam sth.,
        that is not spam, than the other way around.
        The last step is to keep only those words in spam_dict that are not in ham_dict and are not stop_words.
        """

        self.spam_dict = self.get_suspicious_words(X, y, 1)
        ham_dict = self.get_suspicious_words(X, y, 0)
        my_inverted_dict_spam = dict(map(reversed, self.spam_dict.items()))
        my_inverted_dict_ham = dict(map(reversed, ham_dict.items()))
        suspicious_spam = []
        suspicious_ham = []

        for key in my_inverted_dict_spam:
            if key > 15 and my_inverted_dict_spam[key] not in stop:
                suspicious_spam.append(my_inverted_dict_spam[key])

        for key in my_inverted_dict_ham:
            if key > 35 and my_inverted_dict_ham[key] not in stop:
                suspicious_ham.append(my_inverted_dict_ham[key])

        # remove all from spam that is in ham
        self.suspicious_words_list = []
        for word in suspicious_spam:
            if word not in suspicious_ham:
                self.suspicious_words_list.append(word)

        return self

    def transform(self, X, y=None):
        X = X.copy()
        X["HAS_LINK"] = np.where(X['CONTENT'].str.contains('http') |
                                  X['CONTENT'].str.contains('//'), 2, 0)
    
        X["NOT_UNIQUE_AUTHOR"] = np.where(X['AUTHOR'].str in self.not_unique_authors, self.duplicated_and_not_spammers
                                           // self.duplicated_and_spammers, 0)

        X["NULL_IN_DATE_TIME"] = np.where(X["DATE"].isna(), 2, 0)
                
        result = []
        for index, row in X.iterrows():
            suspicious_counter = 0
            for word in self.suspicious_words_list:
                my_row = row["CONTENT"].lower().split()
                if word in my_row:
                    suspicious_counter += self.spam_dict[word.lower()] // 100
            result.append(suspicious_counter)

        X["SUSPICIOUS_WORDS_COUNT"] = result

        return X
    
    def get_suspicious_words(self, X, y, num):
        result = {}
        for index, row in X.iterrows():
            if y[index] != num:
                continue
            words = (row["CONTENT"].lower()).split()
            for word in words:
                if word in result:
                    result[word] += 1
                else:
                    result[word] = 1
                    
        return result

# DEMO
transformed = ExplorativeTransformer().fit_transform(train_X, train_y)
transformed.head()

,COMMENT_ID,AUTHOR,DATE,CONTENT,INTERPRET,HAS_LINK,NOT_UNIQUE_AUTHOR,NULL_IN_DATE_TIME,SUSPICIOUS_WORDS_COUNT
0,z13wxtdpeznid12et23ogtd4zoyvzbnoz04,Sonny Carter,2015-05-22 11:46:35.988,I love this song sooooooooooooooo much﻿,eminem,0,0,0,0
1,_2viQ_Qnc6-adCzTDLAhqNVQ5hFYcjPyPI5m7pHY4BY,Lizzy Molly,2013-09-09 17:34:07.052,PLEASE CHECK OUT MY VIDEO CALLED &quot;WE LOVE...,shakira,0,0,0,3
2,z13uutbriumnuj3rq04ccbvqlwjuj1srhyk0k,Warcorpse666,2015-05-26 02:27:43.254,sorry but eminmem is a worthless wife beating ...,eminem,0,0,0,0
3,z121gbuy2unhc5m4n04cf3kyslqhepeqgvo0k,Santeri Saariokari,2014-09-03 16:32:59.000,"Hey guys go to check my video name ""growtopia ...",katy,0,0,0,3
4,z133hdqrqpukup0lp22chhoaztrhvxov5,Quinho Divulgaçoes,2014-11-06 19:50:16.000,me segue ha https://www.facebook.com/marcos.s...,katy,2,0,0,0


---

## AddMissingDateColumn

In [10]:
class AddMissingDateColumn(BaseEstimator, TransformerMixin):
    """
    Adds boolean column if DATE value is missing.
    The model should then understand that True most likely means spam.
    """
    def __init__(self):
        super().__init__()

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        # do not modify the original dataset
        X = X.copy()
        X["DATE_MISSING"] = X.DATE.isna()
        return X
    
# DEMO
transformed = AddMissingDateColumn().transform(train_X)
transformed[1244:1246]

,COMMENT_ID,AUTHOR,DATE,CONTENT,INTERPRET,DATE_MISSING
1244,_2viQ_Qnc6_FlLJN0izQaKVQNe6LGDmPZMmkVDjjymE,Neeru bala,2013-09-05 23:07:09.056,Hi.. Everyone.. If anyone after real online wo...,shakira,False
1245,LneaDw26bFsltJodWnZAafXscqrATBuKDM8-8lA4TQE,miamiscraziest,NaT,LADIES!!! -----&gt;&gt; If you have a broken h...,eminem,True


---

## AddLongCommentColumn

In [11]:
class AddLongCommentColumn(BaseEstimator, TransformerMixin):
    """
    Adds boolean column if CONTENT is longer or equal to 50 charcters.
    The model should then understand that True most likely means spam.
    """
    def __init__(self):
        super().__init__()

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        X = X.copy()
        X["LONG_COMMENT"] = X.CONTENT.str.len() >= 50
        return X

# DEMO
transformed = AddLongCommentColumn().transform(train_X)
transformed[:2]

,COMMENT_ID,AUTHOR,DATE,CONTENT,INTERPRET,LONG_COMMENT
0,z13wxtdpeznid12et23ogtd4zoyvzbnoz04,Sonny Carter,2015-05-22 11:46:35.988,I love this song sooooooooooooooo much﻿,eminem,False
1,_2viQ_Qnc6-adCzTDLAhqNVQ5hFYcjPyPI5m7pHY4BY,Lizzy Molly,2013-09-09 17:34:07.052,PLEASE CHECK OUT MY VIDEO CALLED &quot;WE LOVE...,shakira,True


---

## AddContainsCheckColumn

In [12]:
class AddContainsCheckColumn(BaseEstimator, TransformerMixin):
    """
    Adds boolean column if CONTENT contains 'check'.
    The model should then understand that True most likely means spam.
    """
    def __init__(self):
        super().__init__()

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        X = X.copy()
        X["CONTAINS_CHECK"] = X.CONTENT.str.contains("check")
        return X

# DEMO
transformed = AddContainsCheckColumn().transform(train_X)
transformed[2:4]

,COMMENT_ID,AUTHOR,DATE,CONTENT,INTERPRET,CONTAINS_CHECK
2,z13uutbriumnuj3rq04ccbvqlwjuj1srhyk0k,Warcorpse666,2015-05-26 02:27:43.254,sorry but eminmem is a worthless wife beating ...,eminem,False
3,z121gbuy2unhc5m4n04cf3kyslqhepeqgvo0k,Santeri Saariokari,2014-09-03 16:32:59.000,"Hey guys go to check my video name ""growtopia ...",katy,True


---

## AddMultipleCommentsColumn

In [13]:
class AddMultipleCommentsColumn(BaseEstimator, TransformerMixin):
    """
    Adds boolean column if author posted multiple comments.
    The model should then understand that True most likely means spam.
    
    Looks also into data it has been fit on.
    """
    def __init__(self):
        super().__init__()

    def fit(self, X, y=None):
        self.prev_X = X.copy()
        return self

    def transform(self, X, y=None):
        X = X.copy()
        joined = pd.concat([X, self.prev_X])
        joined.drop_duplicates(inplace=True)
        X["MULTIPLE_COMMENTS"] = joined.duplicated(subset=["AUTHOR"], keep=False)[:len(X)]
        return X
    
# DEMO
mollys = train_X[train_X.AUTHOR == "Lizzy Molly"]
admc = AddMultipleCommentsColumn()
admc.fit_transform(mollys[:1])

,COMMENT_ID,AUTHOR,DATE,CONTENT,INTERPRET,MULTIPLE_COMMENTS
1,_2viQ_Qnc6-adCzTDLAhqNVQ5hFYcjPyPI5m7pHY4BY,Lizzy Molly,2013-09-09 17:34:07.052,PLEASE CHECK OUT MY VIDEO CALLED &quot;WE LOVE...,shakira,False


In [14]:
transformed = admc.transform(mollys[1:2])
transformed[transformed.AUTHOR == "Lizzy Molly"]

,COMMENT_ID,AUTHOR,DATE,CONTENT,INTERPRET,MULTIPLE_COMMENTS
841,LneaDw26bFuhuiZ8uX6C-qYLIsOFj9BIWtKWtCz870c,Lizzy Molly,NaT,PLEASE CHECK OUT MY VIDEO CALLED &quot;WE LOVE...,eminem,True


---

## AddMultipleCommentsSameVideoColumn

In [15]:
class AddMultipleCommentsSameVideoColumn(BaseEstimator, TransformerMixin):
    """
    Adds boolean column if author posted multiple comments for the same video.
    The model should then understand that True most likely means spam.
    """
    def __init__(self):
        super().__init__()

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        X = X.copy()
        X["MULTIPLE_COMMENTS_SAME_VIDEO"] = X.duplicated(subset=["AUTHOR", "INTERPRET"], keep=False)
        return X
    
# DEMO    
transformed = AddMultipleCommentsSameVideoColumn().transform(train_X)
transformed[transformed.AUTHOR == "Lizzy Molly"]

,COMMENT_ID,AUTHOR,DATE,CONTENT,INTERPRET,MULTIPLE_COMMENTS_SAME_VIDEO
1,_2viQ_Qnc6-adCzTDLAhqNVQ5hFYcjPyPI5m7pHY4BY,Lizzy Molly,2013-09-09 17:34:07.052,PLEASE CHECK OUT MY VIDEO CALLED &quot;WE LOVE...,shakira,False
841,LneaDw26bFuhuiZ8uX6C-qYLIsOFj9BIWtKWtCz870c,Lizzy Molly,NaT,PLEASE CHECK OUT MY VIDEO CALLED &quot;WE LOVE...,eminem,False


---

## AddContainsHttpColumn

In [16]:
class AddContainsHttpColumn(BaseEstimator, TransformerMixin):
    """
    Adds boolean column if CONTENT contains a link.
    The model should then understand that True most likely means spam.
    """
    def __init__(self):
        super().__init__()

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        X = X.copy()
        X["CONTAINS_HTTP"] = X.CONTENT.str.contains("http")
        return X
    
# DEMO    
transformed = AddContainsHttpColumn().transform(train_X)
transformed[3:5]

,COMMENT_ID,AUTHOR,DATE,CONTENT,INTERPRET,CONTAINS_HTTP
3,z121gbuy2unhc5m4n04cf3kyslqhepeqgvo0k,Santeri Saariokari,2014-09-03 16:32:59,"Hey guys go to check my video name ""growtopia ...",katy,False
4,z133hdqrqpukup0lp22chhoaztrhvxov5,Quinho Divulgaçoes,2014-11-06 19:50:16,me segue ha https://www.facebook.com/marcos.s...,katy,True


---

## AddTimeColumn

In [17]:
class AddTimeColumn(BaseEstimator, TransformerMixin):
    """
    Adds time column.
    Hypothesis: spams are posted at night, or, on the contrary, spams are posted during main working hours. Let the model decide...
    This probably won't work, because the dates are most likely relative to the time zone, where they were collected.
    """
    def __init__(self):
        super().__init__()

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        X = X.copy()
        X["TIME"] = X.DATE.dt.time
        return X
    
# DEMO    
transformed = AddTimeColumn().transform(train_X)
transformed[0:1]

,COMMENT_ID,AUTHOR,DATE,CONTENT,INTERPRET,TIME
0,z13wxtdpeznid12et23ogtd4zoyvzbnoz04,Sonny Carter,2015-05-22 11:46:35.988,I love this song sooooooooooooooo much﻿,eminem,11:46:35.988000


---

## HtmlUnescaper

In [18]:
class HtmlUnescaper(BaseEstimator, TransformerMixin):
    """
    For example, `&amp;` is escaped ampersand. Unescape it and other characters as well.
    """
    def __init__(self):
        super().__init__()

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        X = X.copy()
        X["CONTENT"] = X["CONTENT"].apply(html.unescape)
        return X
    
# DEMO    
print("Before: ", train_X.CONTENT[97])
transformed = HtmlUnescaper().transform(train_X)
print("After: ", transformed.CONTENT[97])

Before:  <a href="http://www.youtube.com/watch?v=KQ6zr6kCPj8&amp;t=2m19s">2:19</a> best part﻿
After:  <a href="http://www.youtube.com/watch?v=KQ6zr6kCPj8&t=2m19s">2:19</a> best part﻿


---

## BOMRemover

In [19]:
class BOMRemover(BaseEstimator, TransformerMixin):
    """
    Remove Byte Order Mark from comments.
    """
    def __init__(self):
        super().__init__()

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        X = X.copy()
        X["CONTENT"] = X["CONTENT"].str.replace("\ufeff", "", regex=False)
        return X

# DEMO
"Before: " + train_X.CONTENT.loc[97]

'Before: <a href="http://www.youtube.com/watch?v=KQ6zr6kCPj8&amp;t=2m19s">2:19</a> best part\ufeff'

In [20]:
transformed = BOMRemover().transform(train_X)
"After: " + transformed.CONTENT.loc[97]

'After: <a href="http://www.youtube.com/watch?v=KQ6zr6kCPj8&amp;t=2m19s">2:19</a> best part'

---

## AnchorTransformer

In [21]:
class AnchorTransformer(BaseEstimator, TransformerMixin):
    """
    Transforms all anchor tags into one keyword. 
    The model will figure out, that the presence of this keyword probably means spam.
    """
    def __init__(self):
        super().__init__()

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        X = X.copy()
        X["CONTENT"] = X["CONTENT"].str.replace("<a.+>", "anchortag", regex=True)
        return X
    
# DEMO
print("Before: " + train_X.CONTENT.loc[97])
transformed = AnchorTransformer().transform(train_X)
print("After: " + transformed.CONTENT.loc[97])

Before: <a href="http://www.youtube.com/watch?v=KQ6zr6kCPj8&amp;t=2m19s">2:19</a> best part﻿
After: anchortag best part﻿


---

## AddContainsAnchorTagColumn

In [22]:
class AddContainsAnchorTagColumn(BaseEstimator, TransformerMixin):
    """
    Adds new column CONTAINS_ANCHOR_TAG which is True when CONTENT contains <a> tag.
    The model should then understand that True most likely means spam.
    Removes the link from CONTENT as well.
    """
    def __init__(self):
        super().__init__()

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        X = X.copy()
        X["CONTAINS_ANCHOR_TAG"] = X["CONTENT"].str.contains("<a.+>")
        X["CONTENT"] = X["CONTENT"].str.replace("<a.+>", "", regex=True)
        return X
    
# DEMO
print("Before: " + train_X.CONTENT.loc[97])
transformed = AddContainsAnchorTagColumn().transform(train_X)
print("After:")
transformed.loc[97]

Before: <a href="http://www.youtube.com/watch?v=KQ6zr6kCPj8&amp;t=2m19s">2:19</a> best part﻿
After:


COMMENT_ID             z13uwn2heqndtr5g304ccv5j5kqqzxjadmc0k
AUTHOR                                          Corey Wilson
DATE                              2015-05-28 21:39:52.376000
CONTENT                                           best part﻿
INTERPRET                                              lmfao
CONTAINS_ANCHOR_TAG                                     True
Name: 97, dtype: object

---

## Lower

In [23]:
class Lower(BaseEstimator, TransformerMixin):
    """
    Makes CONTENT lowercase.
    """
    def __init__(self):
        super().__init__()

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        X = X.copy()
        X["CONTENT"] = X["CONTENT"].str.lower()
        return X
    
# DEMO
print("Before: " + train_X.CONTENT.loc[5])
transformed = Lower().transform(train_X)
print("After: " + transformed.CONTENT.loc[5])

Before: Katy Perry is lion﻿
After: katy perry is lion﻿


---

## UrlTransformer

In [24]:
class UrlTransformer(BaseEstimator, TransformerMixin):
    """
    Transforms all urls into one keyword. 
    The model will figure out, that the presence of this keyword probably means spam.
    """
    def __init__(self):
        super().__init__()

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        X = X.copy()
        X["CONTENT"] = X["CONTENT"].str.replace(r"\S*\.com\S*|\S*watch\?\S*", "urllink", regex=True)
        return X
    
# DEMO
print("Before: " + train_X.CONTENT.loc[1220])
print("Before: " + train_X.CONTENT.loc[186])
transformed = UrlTransformer().transform(train_X)
print("After: " + transformed.CONTENT.loc[1220])
print("After: " + transformed.CONTENT.loc[186])

Before: follower please https://www.facebook.com/lists/161620527267482﻿
Before: /watch?v=aImbWbfQbzg watch and subscrible
After: follower please urllink
After: urllink watch and subscrible


---

## Debugger

In [25]:
class Debugger(BaseEstimator, TransformerMixin):
    """
    Prints first 5 rows of dataframe
    """
    def __init__(self):
        super().__init__()

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        print(X[:5])
        return X
    
# DEMO
transformed = Debugger().transform(train_X)

                                    COMMENT_ID              AUTHOR  \
0          z13wxtdpeznid12et23ogtd4zoyvzbnoz04        Sonny Carter   
1  _2viQ_Qnc6-adCzTDLAhqNVQ5hFYcjPyPI5m7pHY4BY         Lizzy Molly   
2        z13uutbriumnuj3rq04ccbvqlwjuj1srhyk0k        Warcorpse666   
3        z121gbuy2unhc5m4n04cf3kyslqhepeqgvo0k  Santeri Saariokari   
4            z133hdqrqpukup0lp22chhoaztrhvxov5  Quinho Divulgaçoes   

                     DATE                                            CONTENT  \
0 2015-05-22 11:46:35.988            I love this song sooooooooooooooo much﻿   
1 2013-09-09 17:34:07.052  PLEASE CHECK OUT MY VIDEO CALLED &quot;WE LOVE...   
2 2015-05-26 02:27:43.254  sorry but eminmem is a worthless wife beating ...   
3 2014-09-03 16:32:59.000  Hey guys go to check my video name "growtopia ...   
4 2014-11-06 19:50:16.000  me segue ha  https://www.facebook.com/marcos.s...   

  INTERPRET  
0    eminem  
1   shakira  
2    eminem  
3      katy  
4      katy  


---

## ContentCountVectorizer

In [32]:
class ContentCountVectorizer(BaseEstimator, TransformerMixin):
    """
    Applies CountVectorizer on `CONTENT` column
    and appends the results to the dataframe.
    """
    def __init__(self):
        super().__init__()
        self.vectorizer = CountVectorizer()

    def fit(self, X, y=None):
        self.vectorizer.fit(X["CONTENT"])
        return self

    def transform(self, X, y=None):
        vectorized = self.vectorizer.transform(X["CONTENT"]) 
        vectorized_df = pd.DataFrame(vectorized.todense(), columns=self.vectorizer.get_feature_names())
        return pd.concat([X, vectorized_df], axis=1)

    
# DEMO
content_vectorizer = ContentCountVectorizer()
content_vectorizer.fit(train_X, train_y)
transformed = content_vectorizer.transform(test_X)
transformed.head()

,COMMENT_ID,AUTHOR,DATE,CONTENT,INTERPRET,00,000,002,018,02,...,ｃｏｍｍｅｎｔ,ｄａｍｎ,ｅｂａｙ,ｆａｎｃy,ｈｔｔｐ,ｉｓ,ｓｈｏｅｃｏｌｌｅｃｔｏｒ314,ｔｈｉｓ,ｕｓｒ,ｗｗｗ
0,z13ozdmr4lf3uzc5z04cix2zkyjzgvcyemw0k,Carlos Thegamer,2013-12-01 01:20:21.000,subscribe to my channel people :D﻿,psy,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,z13aznjg2ojwjlknm23yjbt4uniuepuxs04,Dunrell Pinder,2015-05-23 02:57:54.433,i love her﻿,shakira,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,z12kil5h2ujlyzgnc23qtzqyokqrc3301,Sinaloence LiveUniverseI,2015-04-29 03:58:06.452,I like This Comment and do not kill :P﻿,lmfao,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,z13vhvu54u3ewpp5h04ccb4zuoardrmjlyk0k,Ray Benich,2015-06-05 18:05:16.000,The first billion viewed this because they tho...,psy,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,z12msjqximryxdfz104cjhbhtw2ld5ipsh4,SIR JOHN,2015-05-07 02:28:12.704,EMINEM the best EVER.﻿,eminem,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


---